In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from diffusers import UNet2DModel, DDPMScheduler
from tqdm import tqdm
from accelerate import Accelerator
from pathlib import Path
from fff.data import load_dataset
import numpy as np
import matplotlib.pyplot as plt

2025-11-05 16:17:08.965890: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-05 16:17:09.009734: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-05 16:17:09.778768: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO: Using pytorch backend


In [2]:
class EMA:
    def __init__(self, model, decay=0.9999):
        self.decay = decay
        self.shadow = {}
        self.original = {}
        # Register parameters
        for name, param in model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self, model):
        for name, param in model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = (
                    1.0 - self.decay
                ) * param.data + self.decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def apply_shadow(self, model):
        """Apply EMA weights to model."""
        for name, param in model.named_parameters():
            if name in self.shadow:
                self.original[name] = param.data.clone()
                param.data.copy_(self.shadow[name])

    def restore(self, model):
        """Restore original (non-EMA) weights."""
        for name, param in model.named_parameters():
            if name in self.original:
                param.data.copy_(self.original[name])
        self.original = {}

In [3]:
def train_ct_ddpm(
    dataset_config,
    image_size=384,
    batch_size=32,
    num_epochs=5,
    learning_rate=2e-4,
    save_dir="./flowmatching-chexpert",
    num_train_timesteps=1000,
):
    accelerator = Accelerator()
    device = accelerator.device

    # Load dataset
    train_ds, _, _ = load_dataset(**dataset_config)
    train_loader = DataLoader(
        train_ds, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True
    )

    # Define UNet
    model = UNet2DModel(
        sample_size=image_size,
        in_channels=3,
        out_channels=3,
        layers_per_block=3,
        block_out_channels=(64, 128, 128, 256),
        down_block_types=("DownBlock2D",) * 4,
        up_block_types=("UpBlock2D",) * 4,
    )

    print(
        "Model parameters:",
        sum(p.numel() for p in model.parameters() if p.requires_grad),
    )

    # Noise scheduler
    noise_scheduler = DDPMScheduler.from_pretrained("google/ddpm-cifar10-32") 

    # Optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=num_epochs, eta_min=0
    )
    # Prepare with accelerator
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader)
    model.train()
    ema = EMA(model, decay=0.999)

    loss_history = []
    for epoch in range(num_epochs):
        pbar = tqdm(
            train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", dynamic_ncols=True
        )
        for batch in pbar:
            # Get image: [B, 3, H, W]
            clean_images = (
                batch[0].reshape(batch_size, 3, image_size, image_size).to(device)
            )

            # Sample noise
            noise = torch.randn_like(clean_images)
            timesteps = torch.randint(
                0,
                noise_scheduler.config.num_train_timesteps,
                (clean_images.shape[0],),
                device=clean_images.device,
            ).long()

            # Add noise to image
            noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)


            # Predict the noise using UNet
            noise_pred = model(noisy_images, timesteps).sample

            # Loss
            loss = nn.MSELoss()(noise_pred, noise)
            loss_history.append(loss.item())
            accelerator.backward(loss)
            # gradient clipping
            accelerator.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            lr_scheduler.step()
            ema.update(model)
            optimizer.zero_grad()

            pbar.set_postfix(loss=np.mean(loss_history[-100:]))

        # Save model after each epoch
        if accelerator.is_main_process:
            ema.apply_shadow(model)
            # Now sample or save model here
            model.save_pretrained(Path(save_dir) / f"epoch_{epoch+1}")
            ema.restore(model)

    print("✅ Training complete.")

In [4]:
dataset_config = {
    "name": "chexpert",
    "root": "/mnt/SSD_2TB/cheXpert/",
    "patchsize": None,
    "resize_to": 384,
}
train_ct_ddpm(dataset_config)


/home/armand/miniconda3/lib/python3.10/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Model parameters: 22003907


Epoch 1/5:   0%|                                                                                                                                                                    | 0/5372 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.12 GiB. GPU 0 has a total capacity of 10.75 GiB of which 219.75 MiB is free. Process 1558258 has 2.08 GiB memory in use. Including non-PyTorch memory, this process has 8.44 GiB memory in use. Of the allocated memory 8.21 GiB is allocated by PyTorch, and 47.68 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)